#### Tool Decorators: Keeping Code and Documentation in Sync

Maintaining accurate **tool documentation** is tricky: when a function’s parameters or behavior change, its registered schema and description can easily drift.
A Python **decorator** can keep the function definition and its agent-tool metadata in lockstep.

---

##### The problem

Manual registration:

```python
action_registry.register(Action(
    name="read_file",
    function=read_file,
    description="Reads content from a specified file",
    parameters={"type":"object","properties":{"file_path":{"type":"string"}},
                "required":["file_path"]}
))

def read_file(file_path: str) -> str:
    """Reads and returns the content of a file."""
    with open(file_path) as f:
        return f.read()
```

Adding a new parameter requires updating the schema and description separately—easy to forget.

---

##### Decorator solution

Use a decorator to introspect the function and register it automatically:

```python
@register_tool(tags=["file_operations"])
def read_file(file_path: str) -> str:
    """Reads and returns the content of a file from the given path."""
    with open(file_path) as f:
        return f.read()
```

The decorator:

* takes the function name as the **tool name**
* extracts the **docstring** as description
* builds a **JSON schema** from type hints & signature
* registers the tool and tags in a central registry

---

##### Implementation (core pieces)

```python
def register_tool(tool_name=None, description=None,
                 parameters_override=None, terminal=False, tags=None):
    def decorator(func):
        meta = get_tool_metadata(func, tool_name, description,
                                 parameters_override, terminal, tags)

        tools[meta["tool_name"]] = {
            "description": meta["description"],
            "parameters": meta["parameters"],
            "function": meta["function"],
            "terminal": meta["terminal"],
            "tags": meta["tags"]
        }

        for tag in meta["tags"]:
            tools_by_tag.setdefault(tag, []).append(meta["tool_name"])
        return func
    return decorator
```

Helper to extract metadata:

```python
def get_tool_metadata(func, tool_name=None, description=None,
                      parameters_override=None, terminal=False, tags=None):
    tool_name = tool_name or func.__name__
    description = description or (func.__doc__.strip() if func.__doc__ else "No description provided.")

    if parameters_override is None:
        sig = inspect.signature(func)
        hints = get_type_hints(func)
        args_schema = {"type": "object", "properties": {}, "required": []}

        for name, param in sig.parameters.items():
            if name in ["action_context", "action_agent"]:
                continue
            ptype = hints.get(name, str)
            args_schema["properties"][name] = {"type": get_json_type(ptype)}
            if param.default is inspect.Parameter.empty:
                args_schema["required"].append(name)
    else:
        args_schema = parameters_override

    return {
        "tool_name": tool_name,
        "description": description,
        "parameters": args_schema,
        "function": func,
        "terminal": terminal,
        "tags": tags or []
    }
```

---

##### Why it helps

* **Single source of truth** – docstring and type hints define both code and schema.
* **Automatic updates** – change the function signature or docstring, the tool registration updates itself.
* **Tagging & discovery** – group related tools (`"file_operations"`, `"database_tools"`, …).
* **Cleaner developer experience** – write standard Python functions; the decorator handles agent registration.

---

##### Example change—automatic schema update

```python
@register_tool(tags=["file_operations"])
def read_file(file_path: str, encoding: str = "utf-8") -> str:
    """Reads file content with chosen encoding (default utf-8)."""
    with open(file_path, 'r', encoding=encoding) as f:
        return f.read()
```

Adding `encoding` with a default automatically updates the JSON schema to include it as **optional**—no manual edits required.

---

By letting the **function itself** define its own metadata, decorators keep agent tools, documentation, and parameter schemas perfectly synchronized as the code evolves.


#### Organizing Agent Tools: From Tags to Registries

When building agents, you often create many tools with different purposes—file operations, database queries, API calls, and more.
A simple way to keep them manageable is to organize them with **tags** and **registries**.

This structure has three layers:

1. **Tool decorators** – tag and document individual tools.
2. **Tool registry** – global dictionaries of all tools and their tags.
3. **Action registry** – per-agent selection of tools based on tags.

---

### Tagging Tools for Organization

Decorators attach tags to each tool at definition time:

```python
@register_tool(tags=["file_operations"])
def read_file(file_path: str) -> str:
    """Reads and returns the content of a file."""
    with open(file_path, 'r') as f:
        return f.read()

@register_tool(tags=["file_operations", "write"])
def write_file(file_path: str, content: str) -> None:
    """Writes content to a file."""
    with open(file_path, 'w') as f:
        f.write(content)

@register_tool(tags=["database", "read"])
def query_database(query: str) -> List[Dict]:
    """Executes a database query and returns results."""
    return db.execute(query)
```

During registration the decorator maintains two global dictionaries:

```python
tools = { ... }          # all tools indexed by name
tools_by_tag = {
    "file_operations": ["read_file", "write_file"],
    "write": ["write_file"],
    "database": ["query_database"],
    "read": ["query_database"]
}
```

You can now easily look up all `"file_operations"` tools or all tools tagged `"read"`.

---

### Creating Focused Action Registries

When creating an agent, build an `ActionRegistry` filtered by the tags it needs:

```python
def create_file_processing_agent():
    action_registry = ActionRegistry(tags=["file_operations"])
    return Agent(
        goals=[Goal(1, "File Processing", "Process project files")],
        agent_language=AgentFunctionCallingActionLanguage(),
        action_registry=action_registry,
        generate_response=generate_response,
        environment=Environment()
    )

def create_database_agent():
    action_registry = ActionRegistry(tags=["database"])
    return Agent(
        goals=[Goal(1, "Database Operations", "Query database as needed")],
        agent_language=AgentFunctionCallingActionLanguage(),
        action_registry=action_registry,
        generate_response=generate_response,
        environment=Environment()
    )
```

---

### Specialized Agents by Tag

Specify tags directly to create agents with precise capabilities:

```python
# Read-only agent: can only read, never write
read_only_agent = Agent(
    goals=[Goal(1, "Read Only", "Read but don't modify data")],
    agent_language=AgentFunctionCallingActionLanguage(),
    action_registry=ActionRegistry(tags=["read"]),
    generate_response=generate_response,
    environment=Environment()
)

# Full file-operations agent: read and write files
file_agent = Agent(
    goals=[Goal(1, "File Handler", "Manage file operations")],
    agent_language=AgentFunctionCallingActionLanguage(),
    action_registry=ActionRegistry(tags=["file_operations"]),
    generate_response=generate_response,
    environment=Environment()
)
```

---

### Benefits

* **Clear organization** – tools grouped by purpose and easily discoverable.
* **Selective loading** – each agent gets only the tools it needs.
* **Easier maintenance** – add or remove tools by changing decorators, not agent code.

This tag-and-registry pattern keeps large collections of agent tools manageable and makes specialized agents easy to create.


#### Refactoring the README Agent with Tool Decorators

Manual action registration duplicates effort: you define a function **and** separately maintain its description and parameter schema.
Decorators unify these so the **function itself is the single source of truth**.

---

##### Before

```python
def read_project_file(name: str) -> str:
    with open(name, "r") as f:
        return f.read()

action_registry.register(Action(
    name="read_project_file",
    function=read_project_file,
    description="Reads a file from the project.",
    parameters={
        "type":"object",
        "properties":{"name":{"type":"string"}},
        "required":["name"]
    },
    terminal=False
))
```

Changing the signature or behavior means editing both the function and the registration.

---

##### After: decorator-based tools

```python
@register_tool(tags=["file_operations", "read"])
def read_project_file(name: str) -> str:
    """Reads and returns the content of a specified project file."""
    with open(name, "r") as f:
        return f.read()

@register_tool(tags=["file_operations", "list"])
def list_project_files() -> List[str]:
    """Lists all Python files in the current directory."""
    return sorted([f for f in os.listdir(".") if f.endswith(".py")])

@register_tool(tags=["system"], terminal=True)
def terminate(message: str) -> str:
    """Terminates the agent's execution with a final message."""
    return f"{message}\nTerminating..."
```

The decorator automatically:

* extracts the **docstring** as description,
* builds a **JSON schema** from type hints & signature,
* registers the tool and its **tags**.

---

##### Creating the agent

```python
def main():
    goals = [
        Goal(1, "Gather Information",
             "Read each file to prepare a project README"),
        Goal(1, "Terminate",
             "Call terminate when done and return the README text")
    ]

    agent = Agent(
        goals=goals,
        agent_language=AgentFunctionCallingActionLanguage(),
        action_registry=ActionRegistry(tags=["file_operations", "system"]),
        generate_response=generate_response,
        environment=Environment()
    )

    final_memory = agent.run("Write a README for this project.")
    print(final_memory.get_memories())

if __name__ == "__main__":
    main()
```

`ActionRegistry(tags=[...])` automatically loads only the tagged tools—no manual registration.

---

##### Advantages

* **Self-documenting** – docstrings stay with the code and are shown to the LLM.
* **Automatic parameter inference** – type hints and defaults generate the JSON schema, keeping it in sync.
* **Tag-based organization** – easily group and filter tools (`"file_operations"`, `"system"`, etc.).
* **Simpler agent creation** – specify needed tags; registry does the rest.
* **Easier maintenance** – add, modify, or remove a tool by editing a single function.

---

**Flow**

1. On import, decorators register all tools and their tags.
2. `ActionRegistry(tags=...)` selects the required subset.
3. The agent loop uses these tools without any manual metadata updates.

This refactor eliminates duplicated metadata and keeps your README agent (and any future agents) accurate and easy to extend.
